# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from citipy import citipy

output_data_file = "../output_data/Vacation.csv"


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
airport_df = pd.read_csv("../output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
locations = airport_df[["Latitude", "Longitude"]]
humidity = airport_df["Humidity (%)"].astype(float)
humidity_percent = humidity/100

In [13]:
figure_layout = {
    'width': '1000px',
    'eight': '600px',
    'order': '1px solid black',
    'adding': '1px'}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px', order='1px solid black', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
hotel_df = airport_df[airport_df['Temperature (F)'] < 80]
hotel_df = airport_df[airport_df['Temperature (F)'] > 70]
hotel_df = airport_df[airport_df['Wind Speed (mph)'] < 10]
hotel_df = airport_df[airport_df['Cloudiness (%)'] < 0.001]

hotel_df.to_csv(output_data_file, index=False, header=True)
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Vostok,RU,46.49,135.88,53.29,96,0,1.88
5,East London,ZA,-33.02,27.91,68.70,44,0,5.82
7,Chokurdakh,RU,70.63,147.92,39.43,74,0,12.12
30,Sisimiut,GL,66.94,-53.67,52.59,50,0,10.54
35,Manama,BH,26.22,50.58,95.00,50,0,4.70
...,...,...,...,...,...,...,...,...
532,Henties Bay,NaN,-22.12,14.28,74.80,25,0,12.97
548,Baghlan,AF,36.13,68.71,86.00,16,0,4.97
556,Clarence Town,AU,-32.58,151.78,49.12,93,0,1.12
567,Herat Province,AF,34.50,62.00,89.60,13,0,11.41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Add a "Hotel Name" column to the DataFrame.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df['Hotel Name'] = ""

# target_coordinates = coordinates
# target_search = "City"
# target_radius = 5000
# target_type = "lodging"

for index, row in hotel_df.iterrows():
    coordinates = str(hotel_df.loc[index,'Latitude']) +',' + str(hotel_df.loc[index,'Longitude'])
    params = {
        "location": coordinates,
        "radius": 5000,
        "type": "lodging",
        "key": g_key
        }
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name'];
    except (IndexError):
        continue

hotel_df.to_csv(output_data_file, index=False, header=True)
hotel_df.head()

# set up a parameters dictionary
# run a request using our params dictionary

C:\Users\Mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,Vostok,RU,46.49,135.88,53.29,96,0,1.88,
5,East London,ZA,-33.02,27.91,68.70,44,0,5.82,Tu Casa
7,Chokurdakh,RU,70.63,147.92,39.43,74,0,12.12,
30,Sisimiut,GL,66.94,-53.67,52.59,50,0,10.54,Hotel Sisimiut
35,Manama,BH,26.22,50.58,95.00,50,0,4.70,Al Safir Hotel & Tower


In [16]:
# NOTE: Do not change any of the code in this cell
narrowed_city_df = hotel_df
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
df= pd.read_csv(output_data_file)
locations = df[["Latitude", "Longitude"]]
Humidity = df["Humidity (%)"]

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))